In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark.sql("USE CATALOG demo_project")

print("✅ Ready to build Gold layer!")

In [0]:
# Gold Table 1: Active Customers (Current snapshot)
active_customers = spark.sql("""
    CREATE OR REPLACE TABLE demo_project.gold.active_customers AS
    SELECT 
        customer_id,
        full_name,
        email,
        city,
        subscription_plan,
        account_status,
        valid_from,
        record_version
    FROM demo_project.silver.customers
    WHERE is_current = true 
    AND account_status = 'ACTIVE'
""")

print("✅ Table created: gold.active_customers")
display(spark.table("demo_project.gold.active_customers").limit(10))

In [0]:
# Gold Table 2: Subscription Plan Analytics
plan_metrics = spark.sql("""
    CREATE OR REPLACE TABLE demo_project.gold.plan_metrics AS
    SELECT 
        subscription_plan,
        COUNT(*) as total_customers,
        SUM(CASE WHEN account_status = 'ACTIVE' THEN 1 ELSE 0 END) as active_customers,
        SUM(CASE WHEN account_status = 'INACTIVE' THEN 1 ELSE 0 END) as inactive_customers,
        ROUND(SUM(CASE WHEN account_status = 'ACTIVE' THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) as active_percentage
    FROM demo_project.silver.customers
    WHERE is_current = true
    GROUP BY subscription_plan
    ORDER BY total_customers DESC
""")

print("✅ Table created: gold.plan_metrics")
display(spark.table("demo_project.gold.plan_metrics"))

In [0]:
# Gold Table 3: City-wise Customer Distribution
city_metrics = spark.sql("""
    CREATE OR REPLACE TABLE demo_project.gold.city_metrics AS
    SELECT 
        city,
        COUNT(*) as total_customers,
        COUNT(CASE WHEN subscription_plan = 'Gold' THEN 1 END) as gold_plan_customers,
        COUNT(CASE WHEN subscription_plan = 'Silver' THEN 1 END) as silver_plan_customers,
        COUNT(CASE WHEN subscription_plan = 'Bronze' THEN 1 END) as bronze_plan_customers,
        SUM(CASE WHEN account_status = 'ACTIVE' THEN 1 ELSE 0 END) as active_customers
    FROM demo_project.silver.customers
    WHERE is_current = true
    GROUP BY city
    ORDER BY total_customers DESC
""")

print("✅ Table created: gold.city_metrics")
display(spark.table("demo_project.gold.city_metrics"))

In [0]:
# Gold Table 4: Customers with Multiple Versions (History trackers)
customer_history = spark.sql("""
    CREATE OR REPLACE TABLE demo_project.gold.customer_history AS
    SELECT 
        customer_id,
        full_name,
        COUNT(*) as version_count,
        MIN(valid_from) as first_seen_date,
        MAX(valid_from) as last_updated_date,
        COLLECT_LIST(
            STRUCT(subscription_plan, valid_from, valid_to, is_current)
        ) as plan_history
    FROM demo_project.silver.customers
    GROUP BY customer_id, full_name
    HAVING COUNT(*) > 1
    ORDER BY version_count DESC
""")

print("✅ Table created: gold.customer_history")
print(f"📊 Customers with history: {spark.table('demo_project.gold.customer_history').count()}")
display(spark.table("demo_project.gold.customer_history").limit(10))

In [0]:
# Gold Table 5: Executive Summary (Single row KPIs)
exec_summary = spark.sql("""
    CREATE OR REPLACE TABLE demo_project.gold.executive_summary AS
    SELECT 
        COUNT(DISTINCT customer_id) as total_unique_customers,
        SUM(CASE WHEN is_current = true THEN 1 ELSE 0 END) as current_customers,
        SUM(CASE WHEN is_current = true AND account_status = 'ACTIVE' THEN 1 ELSE 0 END) as active_customers,
        SUM(CASE WHEN is_current = true AND account_status = 'INACTIVE' THEN 1 ELSE 0 END) as inactive_customers,
        SUM(CASE WHEN is_current = true AND subscription_plan = 'Gold' THEN 1 ELSE 0 END) as gold_subscribers,
        SUM(CASE WHEN is_current = true AND subscription_plan = 'Silver' THEN 1 ELSE 0 END) as silver_subscribers,
        SUM(CASE WHEN is_current = true AND subscription_plan = 'Bronze' THEN 1 ELSE 0 END) as bronze_subscribers,
        ROUND(SUM(CASE WHEN is_current = true AND account_status = 'ACTIVE' THEN 1 ELSE 0 END) * 100.0 / 
              SUM(CASE WHEN is_current = true THEN 1 ELSE 0 END), 2) as retention_rate,
        current_date() as report_date
    FROM demo_project.silver.customers
""")

print("✅ Table created: gold.executive_summary")
display(spark.table("demo_project.gold.executive_summary"))

In [0]:
# List all Gold tables
print("=" * 70)
print("📊 GOLD LAYER - ALL TABLES")
print("=" * 70)

gold_tables = spark.sql("SHOW TABLES IN demo_project.gold").filter("isTemporary = false")
display(gold_tables)

print("\n✅ GOLD LAYER COMPLETE! 🎉")
print("\n📋 Gold Tables Created:")
print("  1. active_customers - Current active customer list")
print("  2. plan_metrics - Subscription plan analytics")
print("  3. city_metrics - Geographic distribution")
print("  4. customer_history - Customers with version history")
print("  5. executive_summary - High-level KPIs")